In [1]:
import sys
import importlib
sys.path.append('../PyScripts/')

import utils.function_library 
importlib.reload(utils.function_library)
from utils.function_library import *

### Point 9

#### A preliminary study of larger covmats: As the dimension of the data set increases, ever more covmat elements must be computed. Does this mean each element must be known to better precision to avoid biases (highly probably so)?

In [ ]:
"""
 This is an example for a KiDS1000-like survey with 6 bins instead of 5. The point would be
 to repeat the study with 5,6,7,8,9,10 bins (like Euclid) to study how the increase in 
 dimension affects the bias. 
"""

#Load theory vector
fake_theory_vector = np.reshape(np.load("../Output/xipm/euklids_6_zbins/"
                             "vector_xipm_kids_euklids_6_zbins.npy"), (-1,1))
#Load theory covariance matrix
fake_covmat = np.load("../Output/covmats/CovMatrix_euklids_6_zbins.npy") 

#Create data vector with Euklids characteristics
fake_data_vector = np.random.multivariate_normal(mean=fake_theory_vector[:,0],
                                                 cov=fake_covmat) 

# Dictionary with parameters and their fiducial values
cosmo_params_newthetaI =  {"Omega_m":0.24, "Omega_v":0.69, "sigma_8":0.85,
                           "w0":-1.0, "wa":0.0, "n_spec":0.90, "omb":0.042,
                           "h0":0.68, } 

# Cosmolike inifile for Euklids
euklids_inifile = '../CosmoCov/covs/ini_files/cov_euklids.ini'

# 1 sigma errors(standard deviation)
bestfit_sigmas = K1000_standard_dev_symmetrized()

In [ ]:
# Copy right file into the .ini file that will be called by bash script
subprocess.run(['cp', '../CosmoCov/covs/ini_files/cov_euklids_copy.ini',
            euklids_inifile], stdout=subprocess.PIPE, universal_newlines=True)

# Create X matrix
Xmatrix = create_X_matrix("euklids_6_zbins", fake_theory_vector,
                            euklids_inifile, N_parameters = 8, euklids=True)

np.save("../Output/Xmatrices/X_euklids_6bins.npy", Xmatrix)

In [ ]:
# Case for 5 bins

In [ ]:
X_euklids = np.load("../Output/Xmatrices/X_euklids_5bins.npy")

#compute shift in covariance matrix due to a 5% increase of each matrix element
shift_euklids_5bins = shift_covmat_elements("euklids_5bins",X_euklids, 
                                            fake_covmat, fake_theory_vector, 
                                            fake_data_vector,
                                             cosmo_params_newthetaI, 'up')

In [ ]:
# Case for 6 bins

In [ ]:
X_euklids = np.load("../Output/Xmatrices/X_euklids_6bins.npy")

#compute shift in covariance matrix due to a 5% increase of each matrix element
shift_euklids_6bins = shift_covmat_elements("euklids_6bins",X_euklids, 
                                            fake_covmat, fake_theory_vector, 
                                            fake_data_vector, 
                                            cosmo_params_newthetaI, 'up')


In [ ]:
#Normalize in units of the standard deviation
euklids_bias_5bins = np.array([shift_euklids_5bins[:,:,i]/bestfit_sigmas[j]*100 
                               for i,j in zip([0,2,3,6,7],
                                              range(len(bestfit_sigmas)))])

euklids_bias_6bins = np.array([shift_euklids_6bins[:,:,i]/bestfit_sigmas[j]*100 
                               for i,j in zip([0,2,3,6,7],
                                              range(len(bestfit_sigmas)))])

In [ ]:
np.set_printoptions(suppress=True) #if False, scientific notation on

# Computes the bias for each cosmological parameter (it returns a scalar for easier comparison)
parameter_bias_5bins = euklids_scalar_bias(euklids_bias_5bins)
parameter_bias_5bins

In [ ]:
parameter_bias_6bins =euklids_scalar_bias(euklids_bias_6bins)
parameter_bias_6bins

In [ ]:
# Plot the scalar bias for each simulation 
# ** To be completed with N_zbins=7,8,9,10 (requires a long computational time) **

fig, ax = plt.subplots(1,5, figsize=(12,2), sharex=True)
axes = plt.gca()
axes.yaxis.label.set_size(20)

for i in range(5):
    ax[i].plot([5], [parameter_bias_5bins[i]], '.', markersize=10)
    ax[i].plot([6], [parameter_bias_6bins[i]], '.',  markersize=10)
    
    ax[i].set_xlabel(ylabel_list[i], fontsize=20) 

                   
#fig.subplots_adjust(wspace=0.05, hspace=0.05)
ax[0].set_ylabel(r"$\frac{\theta_{lin} - \hat{\theta}_lin}{\sigma_i}$", 
                 fontsize=20)

plt.subplots_adjust(top = 0.98, bottom = 0.02, right = 1, left = 0, 
                    hspace = 0.07, wspace = 0.05)
fig.tight_layout()
fig.savefig('../Output/plots/point9_scalarbias_zbins.pdf')